# Importing Libraries

In [ ]:
import os
import requests

# from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import subprocess  # For running the Ollama model command

# Load environment & Check for necessary configurations

In [ ]:
# load_dotenv()
ollama_model = "chevalblanc/gpt-4o-mini"

if ollama_model:
    print(f"Using Ollama model '{ollama_model}' for summarization.")
else:
    print("Ollama model not found. Please check your configuration.")

Using Ollama model 'chevalblanc/gpt-4o-mini' for summarization.


# Define Class to represent Webpage

In [3]:
class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        self.title = soup.title.string if soup.title else "There is no title"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

# System prompt and user prompt definitions

In [ ]:
system_prompt = """
You are an assistant tasked with analyzing a website's content and offering a brief summary.
Avoid including text that seems related to navigation. Reply in markdown format.
"""


def user_prompt_for(website):
    user_prompt = f"The Website title is {website.title}\n"
    user_prompt += "The contents of this website are as follows:\nPlease provide a short summary of this website in markdown.\n\n"
    user_prompt += website.text
    return user_prompt

# Prepare messages for the Ollama model

In [5]:
def messages_for(website):
    return f"{system_prompt}\n\n{user_prompt_for(website)}"

# Summarize function using Ollama

In [6]:
def summarize(url):
    website = Website(url)
    prompt = messages_for(website)
    try:
        # Run the Ollama command without --prompt, providing prompt as input
        result = subprocess.run(
            ["ollama", "run", ollama_model],
            input=prompt,  # Send the prompt as standard input
            capture_output=True,
            text=True,
        )
        if result.returncode == 0:
            return result.stdout.strip()
        else:
            return f"Error: {result.stderr}"
    except Exception as e:
        return f"Exception occurred: {str(e)}"

# Display the summary in Markdown format

In [7]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

# Example usage

In [ ]:
# Check if the model is available by trying to run a basic ollama command
try:
    subprocess.run(
        ["ollama", "run", "chevalblanc/gpt-4o-mini", "--help"],
        check=True,
        capture_output=True,
    )
    print("Model is ready to use.")
except subprocess.CalledProcessError:
    print(
        "Model 'chevalblanc/gpt-4o-mini' not found. Please run 'ollama run chevalblanc/gpt-4o-mini' to download it."
    )

Model is ready to use.


In [ ]:
display_summary(
    "https://learn.microsoft.com/en-us/dynamics365/commerce/channel-setup-retail"
)

The code is a part of the documentation for Commerce in Dynamics, specifically about setting up and configuring channels. It provides information on how to set up various types of channels like online channels, call center channels, etc., including prerequisites and steps involved in channel setup.

It also includes links to related topics such as CCPA opt-out, privacy choices, theme settings, previous versions, blog posts, contribution guidelines, and licensing terms.
The documentation is organized into different sections for ease of navigation. It starts with an introduction to the article about setting up channels, followed by a link to the official Dynamics 365 website.

It also includes links to related topics like privacy choices, theme settings, previous versions, blog posts, contribution guidelines, and licensing terms.
Overall, this code provides comprehensive documentation for configuring various types of channels in Commerce.